# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936
Gonçalo Manuel Maia de Sousa A97485

## Problema 1

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    1. Cada reunião ocupa uma sala (enumeradas $$1...S\,$$) durante um “slot”  $$1..T\,$$ $$(\text{hora},\text{dia})$$.  
    2.  Cada reunião tem associado um projeto (enumerados $$1..P$$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $$1..C$$.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $$50\%$$ do total de colaboradores do projeto. 

In [ ]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver('SCIP')

# Criação da matriz de alocação
X = {}
for s in range(S):
    for t in range(T):
        for p in range(P):
            for c in range(C):
                X[s,t,p,c] = horario.BoolVar(f'X[{s}][{t}][{p}][{c}]')

# Construção básica da matriz ^

In [ ]:
# S = Int
# C = Int
# T = (hora, dia)
# P = ([conjunto de colaboradores], lider, número de reuniões semanais)

# Maximizar o número de reuniões efetivamente realizadas
# Minimizar o número médio de reuniões por participante.